In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import time
import datetime
import gzip
import math
from dataclasses import dataclass
from functools import reduce

In [2]:
pattern_loop = r'^\[(.+)\] \[INFO\] ru\.ifmo\.fbsat\.core\.task\.single\.complete\.Cegis\w*: ===== Loop number #(\d+), N = (\d+) =====$'
pattern_optimize_top_down = r'^\[(.+)\] \[INFO\] ru\.ifmo\.fbsat\.core\.task\.Optimization: optimizeTopDown: < (\d+) -> (\d+) in (\d+,\d+) s$'
pattern_cegis_iteration = r'^\[(.+)\] \[DEBUG\] ru\.ifmo\.fbsat\.core\.task\.single\.complete\.Cegis\w*: CEGIS iteration #(\d+) on loop (\d+)$'
pattern_cegis_iteration_done = r'^\[(.+)\] \[INFO\] ru\.ifmo\.fbsat\.core\.task\.single\.complete\.Cegis\w*: CEGIS iteration #(\d+) done in (\d+,\d+) s$'
pattern_cegis_unsat = r'^\[(.+)\] \[INFO\] ru\.ifmo\.fbsat\.core\.task\.single\.complete\.Cegis\w*: UNSAT, N = (\d+) is too small, trying (?:to increase|larger value)...$'
pattern_all_done = r'^\[(.+)\] \[INFO\] ru\.ifmo\.fbsat\.cli\.Main: All done in (\d+,\d+) seconds$'

@dataclass
class TimestampedEvent:
    timestamp: float

@dataclass
class LoopEvent(TimestampedEvent):
    loop: int
    n: int

@dataclass
class OptimizeTopDownEvent(TimestampedEvent):
    from_n: int
    to_n: int
    duration: float

@dataclass
class CegisIterationEvent(TimestampedEvent):
    iteration: int
    loop: int

@dataclass
class CegisIterationDoneEvent(TimestampedEvent):
    iteration: int
    duration: float

@dataclass
class CegisUnsatEvent(TimestampedEvent):
    n: int
        
@dataclass
class AllDone(TimestampedEvent):
    duration: float

def parse_timestamp(s):
    a = datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S,%f")
    return time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S,%f").timetuple()) + 0.000001 * a.microsecond

def parse_duration(s):
    return float(s.replace(',', '.'))

def parse_logs(lines):
    answer = []
    for bline in lines:
        line = bline.decode()
        match = re.search(pattern_loop, line)
        if match:
            timestamp = parse_timestamp(match[1])
            loop = int(match[2])
            n = int(match[3])
            answer.append(LoopEvent(timestamp, loop, n))
            continue
        match = re.search(pattern_optimize_top_down, line)
        if match:
            timestamp = parse_timestamp(match[1])
            from_n = int(match[2])
            to_n = int(match[3])
            duration = parse_duration(match[4])
            answer.append(OptimizeTopDownEvent(timestamp, from_n, to_n, duration))
            continue
        match = re.search(pattern_cegis_iteration, line)
        if match:
            timestamp = parse_timestamp(match[1])
            iteration = int(match[2])
            loop = int(match[3])
            answer.append(CegisIterationEvent(timestamp, iteration, loop))
            continue
        match = re.search(pattern_cegis_iteration_done, line)
        if match:
            timestamp = parse_timestamp(match[1])
            iteration = int(match[2])
            duration = parse_duration(match[3])
            answer.append(CegisIterationDoneEvent(timestamp, iteration, duration))
            continue
        match = re.search(pattern_cegis_unsat, line)
        if match:
            timestamp = parse_timestamp(match[1])
            n = int(match[2])
            answer.append(CegisUnsatEvent(timestamp, n))
            continue
        match = re.search(pattern_all_done, line)
        if match:
            timestamp = parse_timestamp(match[1])
            duration = parse_duration(match[2])
            answer.append(AllDone(timestamp, duration))
            continue
    return answer
        
def parse_log_file(file):
    with gzip.open(file, 'r') as f:
        answer = parse_log(f.readlines())

def read_logs():
    return {file: parse_log(file) for file in os.listdir() if file.startswith('log')}

def get_or_put(d, key, value):
    d.setdefault(key, value)
    return d[key]

def set_dict(d, key, value):
    reduce(lambda dd, k: get_or_put(dd, k, dict()), key[:-1], d)[key[-1]] = value
    
def get_dict(d, key):
    return reduce(lambda dd, k: dd[k], key, d)

def get_stat(logs_pairs, special_mode):
    stats = dict()
    for key in logs_pairs:
        ns = []
        for mode in [special_mode, 'general']:
            with gzip.open("../../" + logs_pairs[key][mode], 'r') as f:
                parsed = parse_logs(f.readlines())
                
            total_duration = None
            need_next_unsat = False
            loop = 0
            iteration = 0;
            n = 0
            ns_i = 0
            unsats = []
            done1s = []
            total_iterations = 0
            n_iterations = dict()

            for event in parsed:
                if isinstance(event, LoopEvent):
                    prev_n = n
                    n = event.n
                    loop = event.loop
                    if mode == special_mode and (len(ns) == 0 or ns[-1] < n):
                        ns.append(n)
                    if ns_i < len(ns) and ns[ns_i] < n:
                        ns_i += 1
                    if prev_n > 0 and prev_n < n:
                        n_iterations[n] = total_iterations
                    continue
                if isinstance(event, OptimizeTopDownEvent):
                    continue
                if isinstance(event, CegisIterationEvent):
                    iteration = event.iteration
                    continue
                if isinstance(event, CegisIterationDoneEvent):
                    if iteration == 1 and ns_i < len(ns) and n == ns[ns_i]:
                        done1s.append(event.timestamp)
                    total_iterations += 1
                    continue
                if isinstance(event, CegisUnsatEvent):
                    if ns_i < len(ns) and n == ns[ns_i]:
                        unsats.append(event.timestamp)
                    continue
                if isinstance(event, AllDone):
                    assert (total_duration is None)
                    total_duration = event.duration
                    continue
                #print(event)
                assert False
                    
            #print(key, mode, n_iterations, total_iterations)
            #if total_duration is None:
                #print(key, mode, n_iterations, total_iterations)
                #print(logs_pairs[key][mode])
            set_dict(stats, key + (f'{mode}_total_duration',), total_duration)
            set_dict(stats, key + (f'{mode}_total_iterations',), total_iterations)
            set_dict(stats, key + (f'{mode}_n_iterations',), n_iterations)
            set_dict(stats, key + (f'{mode}_between_loops_duration_{special_mode}',), [done1s[i] - unsats[i - 1] for i in range(1, len(done1s))])
        #set_dict(stats, key + ('faster_between_loops_duration',), {i + 1: math.log(a) - math.log(g) for i, (a, g) in enumerate(zip(get_dict(stats, key + (f'{special_mode}_between_loops_duration',)), get_dict(stats, key + ('general_between_loops_duration',))))})
        for mode in [special_mode, 'general']:
            set_dict(stats, key + (f'{mode}_between_loops_duration_{special_mode}',), {i + 1: dur for i, dur in enumerate(get_dict(stats, key + (f'{mode}_between_loops_duration_{special_mode}',)))})
        ass_total_dur = get_dict(stats, key + (f'{special_mode}_total_duration',))
        gen_total_dur = get_dict(stats, key + ('general_total_duration',))
        #if ass_total_dur is not None and gen_total_dur is not None:
            #set_dict(stats, key + ('faster_duration',), math.log(ass_total_dur) - math.log(gen_total_dur))
    return stats

pattern_logs_position_line = r'^out/gen/traces_(\d+)_(\d+)_(\d+)\.gz/((?:-|\w|\d)+)/(\d+) (logs/fbsat_\d+_\d+\.log\.gz)$'
def get_logs_pairs():
    with open("../../logs_position") as f:
        params2logs = {((int(match[1]), int(match[2]), int(match[3]), int(match[5])), match[4]): match[6] for match in [re.search(pattern_logs_position_line, line) for line in f.readlines()]}
        num_params = {opts for opts, mode in params2logs}
        return {opts: {"assumptions": params2logs[(opts, "assumptions")], "general": params2logs[(opts, "general1")], "cut_tree": params2logs[(opts, "cut-tree")], "height": params2logs[(opts, "height1")], "bounded": params2logs[(opts, "ass-bounded")]} for opts in num_params if (opts, "assumptions") in params2logs and (opts, "general") in params2logs and (opts, "cut-tree") in params2logs and (opts, "height1") in params2logs and (opts, "ass-bounded") in params2logs}

In [3]:
assumptions_stats = get_stat(get_logs_pairs(), 'assumptions')
cut_tree_stats = get_stat(get_logs_pairs(), 'cut_tree')
height_stats = get_stat(get_logs_pairs(), 'height')
bounded_stats = get_stat(get_logs_pairs(), 'bounded')

In [4]:
def stats_keys(a):
    result = set()
    def stats_keys_internal(b, key):
        for k in b:
            if isinstance(b[k], dict):
                stats_keys_internal(b[k], key + (k,))
            else:
                result.add(key + (k,))
    stats_keys_internal(a, ())
    return result

def merge_stats(a, b):
    d = dict()
    for key in stats_keys(a) & stats_keys(b):
        va = get_dict(a, key)
        vb = get_dict(b, key)
        if va != vb:
            print(key, va, vb)
        assert(va == vb)
        set_dict(d, key, va)
    for key in stats_keys(a) - stats_keys(b):
        set_dict(d, key, get_dict(a, key))
    for key in stats_keys(b) - stats_keys(a):
        set_dict(d, key, get_dict(b, key))
    return d

stats = merge_stats(merge_stats(merge_stats(assumptions_stats, cut_tree_stats), height_stats), bounded_stats)
#print(assumptions_stats)
print(stats[2][18][1][1])

{'general_total_duration': 304.069, 'general_total_iterations': 1308, 'general_n_iterations': {22: 147, 21: 5}, 'height_between_loops_duration_height': {12: 27.175999879837036, 1: 2.246000051498413, 3: 6.171000003814697, 5: 17.588000059127808, 7: 20.10800004005432, 21: 457.7409999370575, 14: 18.49899983406067, 16: 130.77099990844727, 18: 309.34299993515015, 20: 364.9599997997284, 9: 26.526000022888184, 11: 30.06500005722046, 13: 22.48300004005432, 2: 0.9890000820159912, 15: 19.49500012397766, 4: 11.588000059127808, 17: 143.98600006103516, 6: 18.90000009536743, 19: 351.18499994277954, 8: 23.590999841690063, 10: 28.22499990463257}, 'cut_tree_between_loops_duration_cut_tree': {6: 4.859999895095825, 8: 1.4780001640319824, 1: 0.8199999332427979, 17: 7.0849997997283936, 19: 176.78999996185303, 10: 4.3420000076293945, 12: 1.9000000953674316, 14: 2.1589999198913574, 3: 0.25300002098083496, 16: 6.391000032424927, 5: 5.083999872207642, 18: 151.35899996757507, 7: 5.424999952316284, 20: 431.404999

In [5]:
def concat_all(lists):
    res = list()
    for l in lists:
        res = res + l
    return res

def print_to_csv(stats, name):
    import pandas as pd
    lst = []
    for cnt in stats:
        for length in stats[cnt]:
            for test_i in stats[cnt][length]:
                for seed in stats[cnt][length][test_i]:
                    st = stats[cnt][length][test_i][seed]
                    general_total_duration = st['general_total_duration']
                    assumptions_total_duration = st['assumptions_total_duration']
                    cut_tree_total_duration = st['cut_tree_total_duration']
                    height_total_duration = st['height_total_duration']
                    bounded_total_duration = st['bounded_total_duration']
                    general_between_loops_duration = assumptions_stats[cnt][length][test_i][seed]['general_between_loops_duration_assumptions']
                    assumptions_between_loops_duration = assumptions_stats[cnt][length][test_i][seed]['assumptions_between_loops_duration_assumptions']
                    lst.append((cnt, length, test_i, seed, 
                                general_total_duration if general_total_duration is not None else '--', 
                                assumptions_total_duration if assumptions_total_duration is not None else '--',
                                cut_tree_total_duration if cut_tree_total_duration is not None else '--', 
                                height_total_duration if height_total_duration is not None else '--', 
                                bounded_total_duration if bounded_total_duration is not None else '--', 
                                tuple([general_between_loops_duration[i] if i < len(general_between_loops_duration) else '--' for i in range(1, 3)]),
                                tuple([assumptions_between_loops_duration[i] if i < len(assumptions_between_loops_duration) else '--' for i in range(1, 3)])))
    #print(lst)
    lst = sorted(lst)
    sets = [("Число трассировок", [t[0] for t in lst]),
            ("Длина трассировок", [t[1] for t in lst]),
            ("Номер в классе", [t[2] for t in lst]),
            ("Семя SAT-решателя", [t[3] for t in lst]),
            ("Время cegis-min", [t[4] for t in lst]), 
            ("Время cegis-assumptions-min", [t[5] for t in lst]),  
            ("Время cegis-inc-count-min", [t[6] for t in lst]),  
            ("Время cegis-height-min", [t[7] for t in lst]),  
            ("Время cegis-bounded-min", [t[8] for t in lst]),  
           ] + concat_all([[(f"Время между предыдущим SAT и первым UNSAT для cegis-min #{i}", [t[9][i] for t in lst]), 
                            (f"Время между предыдущим SAT и первым UNSAT для cegis-assumptions-min #{i}", [t[10][i] for t in lst]),
                            #(f"Время между предыдущим SAT и первым UNSAT для cegis-inc-count-min #{i}", [t[10][i] for t in lst])
                            #(f"Время между предыдущим SAT и первым UNSAT для cegis-height-min #{i}", [t[11][i] for t in lst])
                           ] 
                           for i in range(2)])
    dictionary = {item[0]: item[1] for item in sets}
    for dk in dictionary:
        for i, value in enumerate(dictionary[dk]):
            if isinstance(value, float):
                dictionary[dk][i] = "{0:9.1f}".format(value)
    pd.DataFrame(dictionary).to_csv(name, index=None, encoding="utf-8")
    
    
print_to_csv(stats, "info.csv")

In [6]:
def extract_info(stats, keys):
    result = dict()
    def extract_info_impl(s, i, k, fk):
        if not isinstance(s, dict):
            result[fk] = s
            return
        for j in s:
            if len(k) != 0 and k[0][0] == i:
                if j in k[0][1]:
                    extract_info_impl(s[j], i + 1, k[1:], fk + (j,))
            else:
                extract_info_impl(s[j], i + 1, k, fk + (j,))
    extract_info_impl(stats, 0, keys, ())
    return result

def split_coordinates(xx, yy, bad=90, splash=0.1):
    assert len(xx) == len(yy)
    x_, y_ = xx, yy
    xx = [x for x, y in zip(x_, y_) if x is not None or y is not None]
    yy = [y for x, y in zip(x_, y_) if x is not None or y is not None]
    nones = len(x_) - len(xx)
    y_xnone = [y for x, y in zip(xx, yy) if x is None]
    x_ynone = [x for x, y in zip(xx, yy) if y is None]
    x_, y_ = xx, yy
    xx = [x for x, y in zip(x_, y_) if x is not None and y is not None]
    yy = [y for x, y in zip(x_, y_) if x is not None and y is not None]
    x_gray = [x for x, y in zip(xx, yy) if y <= bad and x <= bad]
    y_gray = [y for x, y in zip(xx, yy) if y <= bad and x <= bad]
    x_red = [x for x, y in zip(xx, yy) if y > x + splash and (y > bad or x > bad)]
    y_red = [y for x, y in zip(xx, yy) if y > x + splash and (y > bad or x > bad)]
    x_green = [x for x, y in zip(xx, yy) if y < x - splash and (y > bad or x > bad)]
    y_green = [y for x, y in zip(xx, yy) if y < x - splash and (y > bad or x > bad)]
    x_blue = [x for x, y in zip(xx, yy) if y >= x - splash and y <= x + splash and (y > bad or x > bad)]
    y_blue = [y for x, y in zip(xx, yy) if y >= x - splash and y <= x + splash and (y > bad or x > bad)]
    return x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones

def print_N_csv(file, x_green, y_green, x_red, y_red, x_blue, y_blue, opt_name):
    import pandas as pd
    def form(value):
        return "{0:9.2f}".format(value).lstrip().rstrip()
    def make(xs, ys):
        assert len(xs) == len(ys)
        return "".join(map(lambda p: f"({form(p[0])},{form(p[1])})", zip(xs, ys)))
    with open(f'{file}.tex', 'w') as f:
        print("""
\\begin{{tikzpicture}}
\\begin{{axis}}[legend style={{fill opacity=0.8, draw opacity=1,text opacity=1}}, xlabel={{Время синтеза без предположений, с.}}, ylabel={{Время синтеза {}, с.}}]
\\addlegendentry{{Число CEGIS-итераций метода {} меньше({})}}
\\addplot+[only marks,] coordinates {{{}}};
\\addlegendentry{{Число CEGIS-итераций метода cegis-assumptions-min меньше({})}}
\\addplot+[only marks,] coordinates {{{}}};
\\addlegendentry{{Одинаковое число CEGIS-итераций({})}}
\\addplot+[only marks,] coordinates {{{}}};
\\end{{axis}}
\\end{{tikzpicture}}""".format(opt_name.lower(), opt_name, len(x_green), make(x_green, y_green), 
                              len(x_red), make(x_red, y_red), 
                              len(x_blue), make(x_blue, y_blue)), file=f)

def print_loop_csv(file, x_green, y_green, x_red, y_red, x_blue, y_blue, opt_name, gen_name):
    import pandas as pd
    def form(value):
        return "{0:9.2f}".format(value).lstrip().rstrip()
    def make(xs, ys):
        assert len(xs) == len(ys)
        return "".join(map(lambda p: f"({form(p[0])},{form(p[1])})", zip(xs, ys)))
    with open(f'{file}.tex', 'w') as f:
        print("""
\\begin{{tikzpicture}}
\\begin{{axis}}[legend style={{fill opacity=0.8, draw opacity=1,text opacity=1}}, xlabel={{Время синтеза без предположений, с.}}, ylabel={{Время синтеза {}, с.}}]
\\addlegendentry{{{} быстрее({})}}
\\addplot+[only marks,] coordinates {{{}}};
\\addlegendentry{{{} быстрее({})}}
\\addplot+[only marks,] coordinates {{{}}};
\\addlegendentry{{Время различается  менее чем на 0.1с({})}}
\\addplot+[only marks,] coordinates {{{}}};
\\end{{axis}}
\\end{{tikzpicture}}""".format(opt_name.lower(), opt_name, len(x_green), make(x_green, y_green), 
                               gen_name, len(x_red), make(x_red, y_red), 
                               len(x_blue), make(x_blue, y_blue)), file=f)

def print_csv(file, x_green, y_green, x_red, y_red, x_gray, y_gray, y_xnone, x_ynone, nones, opt_name, gen_name):
    import pandas as pd
    def form(value):
        return "{0:9.2f}".format(value).lstrip().rstrip()
    for name, x, y in [(f'{file}-without-faster.csv', x_red, y_red), 
                       (f'{file}-with-faster.csv', x_green, y_green), 
                       (f'{file}-too-fast.csv', x_gray, y_gray),
                       (f'{file}-only-without.csv', x_ynone, [605] * len(x_ynone)),
                       (f'{file}-only-with.csv', [605] * len(y_xnone), y_xnone),
                       (f'{file}-nones.csv', [605] * nones, [605] * nones)]:
        csv = {'x': map(form, x),
               'y': map(form, y)}
        #pd.DataFrame(csv).to_csv(name, index=None, encoding="utf-8")
    def make(xs, ys):
        assert len(xs) == len(ys)
        if len(xs) == 0:
            return "(-10.0, -10.0)"
        return "".join(map(lambda p: f"({form(p[0])},{form(p[1])})", zip(xs, ys)))
    with open(f'{file}.tex', 'w') as f:
        print("""
\\begin{{tikzpicture}}
\\begin{{axis}}[legend style={{fill opacity=0.8, draw opacity=1,text opacity=1}}, xlabel={{Время синтеза без предположений, с.}}, ylabel={{Время синтеза {}, с.}}]
\\addlegendentry{{{} быстрее({})}}
\\addplot+[only marks,] coordinates {{{}}};
\\addlegendentry{{{} быстрее({})}}
\\addplot+[only marks,] coordinates {{{}}};
\\addlegendentry{{Не учтено({})}}
\\addplot+[only marks,] coordinates {{{}}};
\\addlegendentry{{{} не завершился({})}}
\\addplot+[only marks,] coordinates {{{}}};
\\addlegendentry{{{} не завершился({})}}
\\addplot+[only marks,] coordinates {{{}}};
\\addlegendentry{{Оба не завершились({})}}
\\addplot+[only marks,] coordinates {{{}}};
\\addplot[color=black,] {{x}};
\\end{{axis}}
\\end{{tikzpicture}}""".format(opt_name.lower(), opt_name, len(x_green), make(x_green, y_green), 
                               gen_name, len(x_red), make(x_red, y_red), 
                               len(x_gray), make(x_gray, y_gray), 
                               gen_name, len(y_xnone), make([605] * len(y_xnone), y_xnone), 
                               opt_name, len(x_ynone), make(x_ynone, [605] * len(x_ynone)), 
                               nones, make([605] * nones, [605] * nones)), file=f)

In [7]:
if False:
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(assumptions_stats, [(4, {'assumptions_between_loops_duration_assumptions', 'general_between_loops_duration_assumptions'})])
    keys = list({k[:4] + k[5:] for k in info if k[:4] + ('assumptions_between_loops_duration_assumptions',) + k[5:] in info and k[:4] + ('general_between_loops_duration_assumptions',) + k[5:] in info})
    y = [info[k[:4] + ('assumptions_between_loops_duration_assumptions',) + k[4:]] for k in keys]
    x = [info[k[:4] + ('general_between_loops_duration_assumptions'    ,) + k[4:]] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y, 0)
    print_loop_csv('assumptions_loop', x_green, y_green, x_red, y_red, x_blue, y_blue, "cegis-assumptions-min", "cegis-min")

    plt.scatter(x_green, y_green, color='green', label=f'С предположениями быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Без предположений быстрее({len(x_red)})')
    plt.scatter(x_blue, y_blue, color='blue', label=f'Время различается менее чем на 0.1с({len(x_blue)})')
    #plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    mx = max(max(x), max(y))
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    plt.title('Время между предыдущим UNSAT и первым SAT', fontsize=22)
    plt.xlabel('Время синтеза без предположений, с')
    plt.ylabel('Время синтеза с предположениями, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [8]:
if False:
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(cut_tree_stats, [(4, {'cut_tree_between_loops_duration_cut_tree', 'general_between_loops_duration_cut_tree'})])
    keys = list({k[:4] + k[5:] for k in info if k[:4] + ('cut_tree_between_loops_duration_cut_tree',) + k[5:] in info and k[:4] + ('general_between_loops_duration_cut_tree',) + k[5:] in info})
    y = [info[k[:4] + ('cut_tree_between_loops_duration_cut_tree',) + k[4:]] for k in keys]
    x = [info[k[:4] + ('general_between_loops_duration_cut_tree'    ,) + k[4:]] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y, 0)

    plt.scatter(x_green, y_green, color='green', label=f'cut-tree быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Без предположений быстрее({len(x_red)})')
    plt.scatter(x_blue, y_blue, color='blue', label=f'Время различается менее чем на 0.1с({len(x_blue)})')
    #plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    mx = max(max(x), max(y))
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    plt.title('Время между предыдущим UNSAT и первым SAT', fontsize=22)
    plt.xlabel('Время синтеза без предположений, с')
    plt.ylabel('Время синтеза cut-tree, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [9]:
if False:
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(height_stats, [(4, {'height_between_loops_duration_height', 'general_between_loops_duration_height'})])
    keys = list({k[:4] + k[5:] for k in info if k[:4] + ('height_between_loops_duration_height',) + k[5:] in info and k[:4] + ('general_between_loops_duration_height',) + k[5:] in info})
    y = [info[k[:4] + ('height_between_loops_duration_height',) + k[4:]] for k in keys]
    x = [info[k[:4] + ('general_between_loops_duration_height'    ,) + k[4:]] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y, 0)

    plt.scatter(x_green, y_green, color='green', label=f'height быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Без предположений быстрее({len(x_red)})')
    plt.scatter(x_blue, y_blue, color='blue', label=f'Время различается менее чем на 0.1с({len(x_blue)})')
    #plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    mx = max(max(x), max(y))
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    plt.title('Время между предыдущим UNSAT и первым SAT', fontsize=22)
    plt.xlabel('Время синтеза без предположений, с')
    plt.ylabel('Время синтеза height, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [10]:
if False:
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(stats, [(4, {'height_n_iterations', 'assumptions_n_iterations'})])
    keys = list({k[:4] + k[5:] for k in info if k[:4] + ('height_n_iterations',) + k[5:] in info and k[:4] + ('assumptions_n_iterations',) + k[5:] in info})
    y = [info[k[:4] + ('height_n_iterations',) + k[4:]] for k in keys]
    x = [info[k[:4] + ('assumptions_n_iterations',) + k[4:]] for k in keys]
    g = len([i for i in y if i > 3])
    f = len([i for i in y if i > 5])
    o = len([i for i in y if i > 10])
    a = len([i for i in y if i > 100])
    b = len([i for i in y if i > 195])
    c = len([i for i in y if i > 400])
    d = len([i for i in y if i > 800])
    e = len([i for i in y if i > 1400])
    print(len(keys), len(info))
    print(g - f, f - o, o - a, a - b, b - c, c - d, d - e, e)
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y, bad=0)
    print_N_csv('height_n_iterations', x_green, y_green, x_red, y_red, x_blue, y_blue, "cegis-height-min")

    plt.scatter(x_green, y_green, color='green', label=f'Число итераций height меньше({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Число итераций с предположениями меньше({len(x_red)})')
    plt.scatter(x_blue, y_blue, color='blue', label=f'Число итераций одинаковое({len(x_blue)})')
    #plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    mx = max(max(x), max(y))
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    plt.title('Число итераций до нахождения следующего N', fontsize=22)
    plt.xlabel('Число итераций с предположениями, с')
    plt.ylabel('Число итераций height, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [11]:
if False:
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(stats, [(4, {'cut_tree_n_iterations', 'assumptions_n_iterations'})])
    keys = list({k[:4] + k[5:] for k in info if k[:4] + ('cut_tree_n_iterations',) + k[5:] in info and k[:4] + ('assumptions_n_iterations',) + k[5:] in info})
    y = [info[k[:4] + ('cut_tree_n_iterations',) + k[4:]] for k in keys]
    x = [info[k[:4] + ('assumptions_n_iterations',) + k[4:]] for k in keys]
    o = len([i for i in y if i > 10])
    a = len([i for i in y if i > 100])
    b = len([i for i in y if i > 195])
    c = len([i for i in y if i > 400])
    d = len([i for i in y if i > 800])
    e = len([i for i in y if i > 1400])
    print(o - a, a - b, b - c, c - d, d - e, e)
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y, bad=0)
    print_N_csv('cut_tree_n_iterations', x_green, y_green, x_red, y_red, x_blue, y_blue, "cegis-inc-count-min")

    plt.scatter(x_green, y_green, color='green', label=f'Число итераций inc-count меньше({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Число итераций с предположениями меньше({len(x_red)})')
    plt.scatter(x_blue, y_blue, color='blue', label=f'Число итераций одинаковое({len(x_blue)})')
    #plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    mx = max(max(x), max(y))
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    plt.title('Число итераций до нахождения следующего N', fontsize=22)
    plt.xlabel('Число итераций с предположениями, с')
    plt.ylabel('Число итераций height, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [12]:
if False:
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(stats, [(4, {'bounded_n_iterations', 'assumptions_n_iterations'})])
    o = len({i: info[i] for i in info if info[i] > 10})
    a = len({i: info[i] for i in info if info[i] > 100})
    b = len({i: info[i] for i in info if info[i] > 195})
    c = len({i: info[i] for i in info if info[i] > 400})
    print(o - a, a - b, b - c, c)
    keys = list({k[:4] + k[5:] for k in info if k[:4] + ('bounded_n_iterations',) + k[5:] in info and k[:4] + ('assumptions_n_iterations',) + k[5:] in info})
    y = [info[k[:4] + ('bounded_n_iterations',) + k[4:]] for k in keys]
    x = [info[k[:4] + ('assumptions_n_iterations',) + k[4:]] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y, bad=0)
    print_N_csv('bounded_n_iterations', x_green, y_green, x_red, y_red, x_blue, y_blue, "cegis-bounded-min")

    plt.scatter(x_green, y_green, color='green', label=f'Число итераций cegis-bounded-min меньше({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Число итераций cegis-assumptions-min меньше({len(x_red)})')
    plt.scatter(x_blue, y_blue, color='blue', label=f'Число итераций одинаковое({len(x_blue)})')
    #plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    mx = max(max(x), max(y))
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    plt.title('Число итераций до нахождения следующего N', fontsize=22)
    plt.xlabel('Число итераций cegis-assumptions-min, с')
    plt.ylabel('Число итераций cegis-bounded-min, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [13]:
if False:
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(assumptions_stats, [(4, {'assumptions_total_duration', 'general_total_duration'})])
    #keys = list({k[:4] for k in info if info[k[:4] + ('assumptions_total_duration',)] is not None and info[k[:4] + ('general_total_duration',)] is not None})
    keys = list({k[:4] for k in info})
    y = [info[k[:4] + ('assumptions_total_duration',)] for k in keys]
    x = [info[k[:4] + ('general_total_duration'    ,)] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y)
    mx = max(x_green + y_green + x_red + y_red + x_blue + y_blue + x_gray + y_gray)
    print_csv('assumptions', x_green, y_green, x_red, y_red, x_gray, y_gray, y_xnone, x_ynone, nones, "cegis-assumptions-min", "cegis-min")

    plt.scatter(x_green, y_green, color='green', label=f'С предположениями быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Без предположений быстрее({len(x_red)})')
    #plt.scatter(x_blue, y_blue, color='blue', label=f'Одинаково быстро({len(x_blue)})')
    plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    plt.scatter([mx + 5] * len(y_xnone), y_xnone, color='pink', label=f'Без предположений не завершились({len(y_xnone)})')
    plt.scatter(x_ynone, [mx + 5] * len(x_ynone), color='black', label=f'С предположениями не завершились({len(x_ynone)})')
    plt.scatter([], [], color='yellow', label=f'Оба не завершились({nones})')
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    plt.title('Время синтеза', fontsize=22)
    plt.xlabel('Время синтеза без предположений, с')
    plt.ylabel('Время синтеза с предположениями, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [14]:
if False:
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(stats, [(4, {'cut_tree_total_duration', 'assumptions_total_duration'})])
    keys = list({k[:4] for k in info})
    y = [info[k[:4] + ('cut_tree_total_duration',)] for k in keys]
    x = [info[k[:4] + ('assumptions_total_duration',)] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y)
    mx = max(x_green + y_green + x_red + y_red + x_blue + y_blue + x_gray + y_gray)
    print_csv('count', x_green, y_green, x_red, y_red, x_gray, y_gray, y_xnone, x_ynone, nones, "cegis-inc-count-min", "cegis-assumptions-min")

    plt.scatter(x_green, y_green, color='green', label=f'Количественная эвристика быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Без предположений быстрее({len(x_red)})')
    #plt.scatter(x_blue, y_blue, color='blue', label=f'Одинаково быстро({len(x_blue)})')
    plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    plt.scatter([mx + 5] * len(y_xnone), y_xnone, color='pink', label=f'Без предположений не завершились({len(y_xnone)})')
    plt.scatter(x_ynone, [mx + 5] * len(x_ynone), color='black', label=f'Количественная эвристика не завершилась({len(x_ynone)})')
    plt.scatter([], [], color='yellow', label=f'Оба не завершились({nones})')
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    #plt.title('Время синтеза', fontsize=22)
    plt.xlabel('Время синтеза без предположений, с')
    plt.ylabel('Время синтеза количественной эвристики, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [15]:
if False:
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(stats, [(4, {'height_total_duration', 'assumptions_total_duration'})])
    keys = list({k[:4] for k in info})
    y = [info[k[:4] + ('height_total_duration' ,)] for k in keys]
    x = [info[k[:4] + ('assumptions_total_duration',)] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y)
    mx = max(x_green + y_green + x_red + y_red + x_blue + y_blue + x_gray + y_gray)
    print_csv('height', x_green, y_green, x_red, y_red, x_gray, y_gray, y_xnone, x_ynone, nones, "cegis-height-min", "cegis-assumptions-min")

    plt.scatter(x_green, y_green, color='green', label=f'Высотная эвристика быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Без предположений быстрее({len(x_red)})')
    #plt.scatter(x_blue, y_blue, color='blue', label=f'Одинаково быстро({len(x_blue)})')
    plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    plt.scatter([mx + 5] * len(y_xnone), y_xnone, color='pink', label=f'Без предположений не завершились({len(y_xnone)})')
    plt.scatter(x_ynone, [mx + 5] * len(x_ynone), color='black', label=f'Высотная эвристика не завершилась({len(x_ynone)})')
    plt.scatter([], [], color='yellow', label=f'Оба не завершились({nones})')
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    #plt.title('Время синтеза', fontsize=22)
    plt.xlabel('Время синтеза без предположений, с')
    plt.ylabel('Время синтеза высотной эвристики, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [16]:
if False:
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(stats, [(4, {'bounded_total_duration', 'assumptions_total_duration'})])
    keys = list({k[:4] for k in info})
    y = [info[k[:4] + ('bounded_total_duration' ,)] for k in keys]
    x = [info[k[:4] + ('assumptions_total_duration',)] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y)
    mx = max(x_green + y_green + x_red + y_red + x_blue + y_blue + x_gray + y_gray)
    print_csv('bounded', x_green, y_green, x_red, y_red, x_gray, y_gray, y_xnone, x_ynone, nones, "cegis-bounded-min", "cegis-assumptions-min")

    plt.scatter(x_green, y_green, color='green', label=f'Высотная эвристика быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Без предположений быстрее({len(x_red)})')
    #plt.scatter(x_blue, y_blue, color='blue', label=f'Одинаково быстро({len(x_blue)})')
    plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    plt.scatter([mx + 5] * len(y_xnone), y_xnone, color='pink', label=f'Без предположений не завершились({len(y_xnone)})')
    plt.scatter(x_ynone, [mx + 5] * len(x_ynone), color='black', label=f'Высотная эвристика не завершилась({len(x_ynone)})')
    plt.scatter([], [], color='yellow', label=f'Оба не завершились({nones})')
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    #plt.title('Время синтеза', fontsize=22)
    plt.xlabel('Время синтеза без предположений, с')
    plt.ylabel('Время синтеза высотной эвристики, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [17]:
if False:
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(stats, [(4, {'bounded_total_duration', 'cut_tree_total_duration'})])
    keys = list({k[:4] for k in info})
    y = [info[k[:4] + ('bounded_total_duration' ,)] for k in keys]
    x = [info[k[:4] + ('cut_tree_total_duration',)] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y)
    mx = max(x_green + y_green + x_red + y_red + x_blue + y_blue + x_gray + y_gray)
    print_csv('bounded-vs-count', x_green, y_green, x_red, y_red, x_gray, y_gray, y_xnone, x_ynone, nones, "cegis-bounded-min", "cegis-inc-count-min")

    plt.scatter(x_green, y_green, color='green', label=f'bounded быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'inc-count быстрее({len(x_red)})')
    #plt.scatter(x_blue, y_blue, color='blue', label=f'Одинаково быстро({len(x_blue)})')
    plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    plt.scatter([mx + 5] * len(y_xnone), y_xnone, color='pink', label=f'cut-tree не завершились({len(y_xnone)})')
    plt.scatter(x_ynone, [mx + 5] * len(x_ynone), color='black', label=f'bounded не завершилась({len(x_ynone)})')
    plt.scatter([], [], color='yellow', label=f'Оба не завершились({nones})')
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    #plt.title('Время синтеза', fontsize=22)
    plt.xlabel('Время синтеза cut-tree, с')
    plt.ylabel('Время синтеза bounded, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [18]:
if False:
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(stats, [(4, {'bounded_total_duration', 'height_total_duration'})])
    keys = list({k[:4] for k in info})
    y = [info[k[:4] + ('bounded_total_duration' ,)] for k in keys]
    x = [info[k[:4] + ('height_total_duration',)] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y)
    mx = max(x_green + y_green + x_red + y_red + x_blue + y_blue + x_gray + y_gray)
    print_csv('bounded-vs-height', x_green, y_green, x_red, y_red, x_gray, y_gray, y_xnone, x_ynone, nones, "cegis-bounded-min", "cegis-height-min")

    plt.scatter(x_green, y_green, color='green', label=f'bounded быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'height быстрее({len(x_red)})')
    #plt.scatter(x_blue, y_blue, color='blue', label=f'Одинаково быстро({len(x_blue)})')
    plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    plt.scatter([mx + 5] * len(y_xnone), y_xnone, color='pink', label=f'height не завершились({len(y_xnone)})')
    plt.scatter(x_ynone, [mx + 5] * len(x_ynone), color='black', label=f'bounded не завершилась({len(x_ynone)})')
    plt.scatter([], [], color='yellow', label=f'Оба не завершились({nones})')
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    #plt.title('Время синтеза', fontsize=22)
    plt.xlabel('Время синтеза height, с')
    plt.ylabel('Время синтеза bounded, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [19]:
for i in range(1, 100):
    continue
    
    plt.figure(figsize=(8,6), dpi=300)

    info = extract_info(assumptions_stats, [(4, {'assumptions_between_loops_duration', 'general_between_loops_duration'}), (5, {i})])
    if len(info) == 0:
        break
    keys = list({k[:4] + k[5:] for k in info if k[:4] + ('assumptions_between_loops_duration',) + k[5:] in info and k[:4] + ('general_between_loops_duration',) + k[5:] in info})
    y = [info[k[:4] + ('assumptions_between_loops_duration',) + k[4:]] for k in keys]
    x = [info[k[:4] + ('general_between_loops_duration'    ,) + k[4:]] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray = split_coordinates(x, y, 0)
    
    plt.scatter(x_green, y_green, color='green', label=f'С предположениями быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Без предположений быстрее({len(x_red)})')
    plt.scatter(x_blue, y_blue, color='blue', label=f'Одинаково быстро({len(x_blue)})')
    #plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    mx = max(max(x), max(y))
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')
    
    plt.title(f'durations-between-loops on iteration-number={i}', fontsize=22)
    plt.xlabel('Время синтеза без предположений, с')
    plt.ylabel('Время синтеза с предположениями, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [20]:
for i in range(18, 21):
    continue
    
    plt.figure(figsize=(8,6), dpi=300)
    info = extract_info(assumptions_stats, [(1, {i}), (4, {'assumptions_between_loops_duration', 'general_between_loops_duration'})])
    keys = list({k[:4] + k[5:] for k in info if k[:4] + ('assumptions_between_loops_duration',) + k[5:] in info and k[:4] + ('general_between_loops_duration',) + k[5:] in info})
    y = [info[k[:4] + ('assumptions_between_loops_duration',) + k[4:]] for k in keys]
    x = [info[k[:4] + ('general_between_loops_duration'    ,) + k[4:]] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray = split_coordinates(x, y, 0)
    
    plt.scatter(x_green, y_green, color='green', label=f'С предположениями быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Без предположений быстрее({len(x_red)})')
    plt.scatter(x_blue, y_blue, color='blue', label=f'Время различается менее чем на 0.1с({len(x_blue)})')
    #plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    mx = max(max(x), max(y))
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')
    
    plt.title(f'durations-between-loops on trace-length={i}', fontsize=22)
    plt.xlabel('сек. без предположений')
    plt.ylabel('сек. с предположениями')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()


In [21]:
for i in range(18, 21):
    continue 
    
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(assumptions_stats, [(1, {i}), (4, {'assumptions_total_duration', 'general_total_duration'})])
    keys = list({k[:4] for k in info})
    y = [info[k[:4] + ('assumptions_total_duration',)] for k in keys]
    x = [info[k[:4] + ('general_total_duration'    ,)] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y)
    mx = max(x_green + y_green + x_red + y_red + x_blue + y_blue + x_gray + y_gray)
    print_csv(f'assumptions-{i}', x_green, y_green, x_red, y_red, x_gray, y_gray, y_xnone, x_ynone, nones, "cegis-assumptions-min", "cegis-min")

    plt.scatter(x_green, y_green, color='green', label=f'С предположениями быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Без предположений быстрее({len(x_red)})')
    #plt.scatter(x_blue, y_blue, color='blue', label=f'Одинаково быстро({len(x_blue)})')
    plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    plt.scatter([mx + 5] * len(y_xnone), y_xnone, color='pink', label=f'Без предположений не завершились({len(y_xnone)})')
    plt.scatter(x_ynone, [mx + 5] * len(x_ynone), color='black', label=f'С предположениями не завершились({len(x_ynone)})')
    plt.scatter([], [], color='yellow', label=f'Оба не завершились({nones})')
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    #plt.title(f'Время синтеза с длиной трассировки {i}', fontsize=20)
    plt.xlabel('Время синтеза без предположений, с')
    plt.ylabel('Время синтеза с предположениями, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [22]:
for i in range(18, 21):
    continue 
    
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(stats, [(1, {i}), (4, {'cut_tree_total_duration', 'assumptions_total_duration'})])
    keys = list({k[:4] for k in info})
    y = [info[k[:4] + ('cut_tree_total_duration',)] for k in keys]
    x = [info[k[:4] + ('assumptions_total_duration'    ,)] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y)
    mx = max(x_green + y_green + x_red + y_red + x_blue + y_blue + x_gray + y_gray)
    print_csv(f'count-{i}', x_green, y_green, x_red, y_red, x_gray, y_gray, y_xnone, x_ynone, nones, "cegis-inc-count-min", "cegis-assumptions-min")

    plt.scatter(x_green, y_green, color='green', label=f'Количественная эвристика быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Без предположений быстрее({len(x_red)})')
    #plt.scatter(x_blue, y_blue, color='blue', label=f'Одинаково быстро({len(x_blue)})')
    plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    plt.scatter([mx + 5] * len(y_xnone), y_xnone, color='pink', label=f'Без предположений не завершились({len(y_xnone)})')
    plt.scatter(x_ynone, [mx + 5] * len(x_ynone), color='black', label=f'Количественная эвристика не завершилась({len(x_ynone)})')
    plt.scatter([], [], color='yellow', label=f'Оба не завершились({nones})')
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    #plt.title(f'Время синтеза с длиной трассировки {i}', fontsize=20)
    plt.xlabel('Время синтеза без предположений, с')
    plt.ylabel('Время синтеза количественной эвристики, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [23]:
for i in range(18, 21):
    continue
    
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(stats, [(1, {i}), (4, {'height_total_duration', 'assumptions_total_duration'})])
    keys = list({k[:4] for k in info})
    y = [info[k[:4] + ('height_total_duration',)] for k in keys]
    x = [info[k[:4] + ('assumptions_total_duration',)] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y)
    mx = max(x_green + y_green + x_red + y_red + x_blue + y_blue + x_gray + y_gray)
    print_csv(f'height-{i}', x_green, y_green, x_red, y_red, x_gray, y_gray, y_xnone, x_ynone, nones, "cegis-height-min", "cegis-assumptions-min")

    plt.scatter(x_green, y_green, color='green', label=f'Высотная эвристика быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Без предположений быстрее({len(x_red)})')
    #plt.scatter(x_blue, y_blue, color='blue', label=f'Одинаково быстро({len(x_blue)})')
    plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    plt.scatter([mx + 5] * len(y_xnone), y_xnone, color='pink', label=f'Без предположений не завершились({len(y_xnone)})')
    plt.scatter(x_ynone, [mx + 5] * len(x_ynone), color='black', label=f'Высотная эвристика не завершилaсь({len(x_ynone)})')
    plt.scatter([], [], color='yellow', label=f'Оба не завершились({nones})')
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    #plt.title(f'Время синтеза с длиной трассировки {i}', fontsize=20)
    plt.xlabel('Время синтеза без предположений, с')
    plt.ylabel('Время синтеза высотной эвристики, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [24]:
for i in range(18, 21):
    continue
    
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(stats, [(1, {i}), (4, {'bounded_total_duration', 'assumptions_total_duration'})])
    keys = list({k[:4] for k in info})
    y = [info[k[:4] + ('bounded_total_duration',)] for k in keys]
    x = [info[k[:4] + ('assumptions_total_duration',)] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y)
    mx = max(x_green + y_green + x_red + y_red + x_blue + y_blue + x_gray + y_gray)
    print_csv(f'bounded-{i}', x_green, y_green, x_red, y_red, x_gray, y_gray, y_xnone, x_ynone, nones, "cegis-bounded-min", "cegis-assumptions-min")

    plt.scatter(x_green, y_green, color='green', label=f'Высотная эвристика быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Без предположений быстрее({len(x_red)})')
    #plt.scatter(x_blue, y_blue, color='blue', label=f'Одинаково быстро({len(x_blue)})')
    plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    plt.scatter([mx + 5] * len(y_xnone), y_xnone, color='pink', label=f'Без предположений не завершились({len(y_xnone)})')
    plt.scatter(x_ynone, [mx + 5] * len(x_ynone), color='black', label=f'Высотная эвристика не завершилaсь({len(x_ynone)})')
    plt.scatter([], [], color='yellow', label=f'Оба не завершились({nones})')
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    #plt.title(f'Время синтеза с длиной трассировки {i}', fontsize=20)
    plt.xlabel('Время синтеза без предположений, с')
    plt.ylabel('Время синтеза высотной эвристики, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [25]:
for i in range(18, 21):
    continue 
    
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(stats, [(1, {i}), (4, {'bounded_total_duration', 'cut_tree_total_duration'})])
    keys = list({k[:4] for k in info})
    y = [info[k[:4] + ('bounded_total_duration',)] for k in keys]
    x = [info[k[:4] + ('cut_tree_total_duration',)] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y)
    mx = max(x_green + y_green + x_red + y_red + x_blue + y_blue + x_gray + y_gray)
    print_csv(f'bounded-vs-count-{i}', x_green, y_green, x_red, y_red, x_gray, y_gray, y_xnone, x_ynone, nones, "cegis-bounded-min", "cegis-inc-count-min")

    plt.scatter(x_green, y_green, color='green', label=f'Высотная эвристика быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Без предположений быстрее({len(x_red)})')
    #plt.scatter(x_blue, y_blue, color='blue', label=f'Одинаково быстро({len(x_blue)})')
    plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    plt.scatter([mx + 5] * len(y_xnone), y_xnone, color='pink', label=f'Без предположений не завершились({len(y_xnone)})')
    plt.scatter(x_ynone, [mx + 5] * len(x_ynone), color='black', label=f'Высотная эвристика не завершилaсь({len(x_ynone)})')
    plt.scatter([], [], color='yellow', label=f'Оба не завершились({nones})')
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    #plt.title(f'Время синтеза с длиной трассировки {i}', fontsize=20)
    plt.xlabel('Время синтеза без предположений, с')
    plt.ylabel('Время синтеза высотной эвристики, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [26]:
for i in range(18, 21):
    continue
    
    plt.figure(figsize=(8,6), dpi= 300)

    info = extract_info(stats, [(1, {i}), (4, {'bounded_total_duration', 'height_total_duration'})])
    keys = list({k[:4] for k in info})
    y = [info[k[:4] + ('bounded_total_duration',)] for k in keys]
    x = [info[k[:4] + ('height_total_duration',)] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray, y_xnone, x_ynone, nones = split_coordinates(x, y)
    mx = max(x_green + y_green + x_red + y_red + x_blue + y_blue + x_gray + y_gray)
    print_csv(f'bounded-vs-height-{i}', x_green, y_green, x_red, y_red, x_gray, y_gray, y_xnone, x_ynone, nones, "cegis-bounded-min", "cegis-height-min")

    plt.scatter(x_green, y_green, color='green', label=f'Высотная эвристика быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Без предположений быстрее({len(x_red)})')
    #plt.scatter(x_blue, y_blue, color='blue', label=f'Одинаково быстро({len(x_blue)})')
    plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    plt.scatter([mx + 5] * len(y_xnone), y_xnone, color='pink', label=f'Без предположений не завершились({len(y_xnone)})')
    plt.scatter(x_ynone, [mx + 5] * len(x_ynone), color='black', label=f'Высотная эвристика не завершилaсь({len(x_ynone)})')
    plt.scatter([], [], color='yellow', label=f'Оба не завершились({nones})')
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    #plt.title(f'Время синтеза с длиной трассировки {i}', fontsize=20)
    plt.xlabel('Время синтеза без предположений, с')
    plt.ylabel('Время синтеза высотной эвристики, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()

In [27]:
for i in range(1, 11):
    continue
    
    plt.figure(figsize=(8,6), dpi= 300)
    
    info = extract_info(stats, [(0, {i}), (4, {'assumptions_total_duration', 'general_total_duration'})])
    keys = list({k[:4] for k in info if info[k[:4] + ('assumptions_total_duration',)] is not None and info[k[:4] + ('general_total_duration',)] is not None})
    y = [info[k[:4] + ('assumptions_total_duration',)] for k in keys]
    x = [info[k[:4] + ('general_total_duration'    ,)] for k in keys]
    x_green, y_green, x_red, y_red, x_blue, y_blue, x_gray, y_gray = split_coordinates(x, y)

    plt.scatter(x_green, y_green, color='green', label=f'С предположениями быстрее({len(x_green)})')
    plt.scatter(x_red, y_red, color='red', label=f'Без предположений быстрее({len(x_red)})')
    #plt.scatter(x_blue, y_blue, color='blue', label=f'Одинаково быстро({len(x_blue)})')
    plt.scatter(x_gray, y_gray, color='gray', label=f'Не учтены({len(x_gray)})')
    mx = max(max(x), max(y))
    plt.plot([-3, mx + 10], [-3, mx + 10])
    plt.axis([-3, mx + 10, -3, mx + 10], 'equal')

    plt.title(f'Время с числом трассировок {i}', fontsize=22)
    plt.xlabel('Время синтеза без предположений, с')
    plt.ylabel('Время синтеза с предположениями, с')
    plt.grid()
    plt.gca().set_aspect("equal")
    plt.legend()
    plt.show()